# Quantum Phase Estimation

The Quantum Phase Estimation (QPE) algorithm estimates the phase (or eigenvalue) of a unitary operator. More precisely, given a unitary operator $U$, and a quantum state $|ψ⟩$, the algorithm estimates $θ$ in $U|ψ⟩=e^{2 \pi iθ}|ψ⟩$.

Quantum phase estimation plays a key role in quantum computing as it is subroutine of, for example, Shor's algorithm. 

## Fundamentals

The circuit, shown hereafter, is made of two qubit registers, the upper $n$ qubits register, and the lower $m$ qubits $|ψ⟩$ state register.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/PhaseCircuit-crop.svg/750px-PhaseCircuit-crop.svg.png" width=500 height=206 />
<center> Image source: Wikipedia

The QPE algorithm uses phase kickback to write the phase of $U$ to the first $n$ qubits register. The inverse QFT is then used to allow a measurement in the computational basis on the first register.

First and second registers are initialized in $|0⟩^{⊗n}$ and $|ψ⟩$ states respectively, then QPE steps are as follow:

- Apply n-bit Hadamard gate operation $H^{⊗n}$ on the first register.
- Apply controlled unitary $CU$ operations. $CU$ is a controlled-U gate which applies the unitary operator $U$ on the second register only if its corresponding control bit (from the first register) is $|1⟩$.
- Apply inverse Quantum Fourier Transform on the first register.
- Measure the first register.

To estimate the phase from a measurement result, we use the following formula:
$$ \theta = \frac{M}{2^n} $$
Where $M$ is the measurement result and $n$ is the number of qubits in the first register.

## Implementation

Now we will implement the Quantum Phase Estimation algorithm. To do so, we define a 4 qubits circuit, where the 3 first qubits are regarding the first register of the previous section, and the fourth qubit is the $|ψ⟩$ state register.

As an example, we will take the $S$ gate, and estimate its phase using the QPE algorithm.

The $S$ gate adds a phase of $\frac{\pi}{2}$ to the state $|1⟩$:

$$S|1⟩ = \begin{bmatrix} 1 & 0 \\ 0 & e^{i \frac{\pi}{2}} \end{bmatrix}\begin{bmatrix} 0 \\ 1 \end{bmatrix} = e^{i \frac{\pi}{2}}|1⟩ $$

As QPE will give us $\theta$ where:

$$S|1⟩ = e^{2i\pi\theta}|1⟩ $$

QPE should return:
$$\theta = \frac{1}{4} $$

#### Let's code

Import python modules

In [1]:
from qat.lang.AQASM import Program, H, X, S
from qat.lang.AQASM.qftarith import QFT
from qat.qpus import PyLinalg

Create the program

In [2]:
qpe = Program()

qbits = qpe.qalloc(4)
cbits = qpe.calloc(3)

Initialize the state $|ψ⟩=|1⟩$ by applying a $X$ gate

In [3]:
X(qbits[3])

circuit = qpe.to_circ()
%qatdisplay circuit --svg

Apply Hadamard gate on three first qubits

In [4]:
for qbit in range(3):
    H(qbits[qbit])

circuit = qpe.to_circ()
%qatdisplay circuit --svg

Apply controlled unitary $CU$ operations

In [5]:
reps = 1
for iqubit in range(3):
    for i in range(reps):
        S.ctrl()(qbits[iqubit], qbits[3])
    reps *= 2

circuit = qpe.to_circ()
%qatdisplay circuit --svg

Apply inverse Quantum Fourier Transform

In [6]:
QFT(3).dag()(qbits[:3])

circuit = qpe.to_circ()
%qatdisplay circuit --svg

Finally, simulate the circuit

In [7]:
circuit = qpe.to_circ()

result = PyLinalg().submit(circuit.to_job(qubits=[0,1,2], nbshots=100))

for sample in result:
    print("State %s, probability %s"%(sample.state, sample.probability))

State |010>, probability 1.0


We obtain state $|010⟩$ with probability 1, state $|010⟩$ equals decimal 2, by applying the previous formula, we have:
$$\theta = \frac{2}{2^{3}}=\frac{2}{8}=\frac{1}{4} $$

Which is the expected phase.